In [8]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset #IterableDataset
from tqdm import tqdm

import sys
import os

os.chdir(os.path.abspath(''))
sys.path.append(os.path.abspath(os.path.abspath('')))
sys.path.append(os.path.abspath(os.path.abspath('') + '/src'))

from utils import *
from models import *

import pandas as pd
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
df_train_total = pd.read_csv("../data/df_train_total.csv")
df_test_total = pd.read_csv("../data/df_test_total.csv")
df_merged = pd.read_csv("../data/df_merged.csv")

In [10]:
train_input_deng, train_label = generate_ts_data_for_deng(df_train_total, df_merged)

def train(model, loader, criterion, optimizer, device):
    
    model.train()
    
    total_loss = []
    
    for batch in loader:
        conti_input, true_y = batch 
        
        conti_input = conti_input.to(device)
        true_y = true_y.to(device)
        
        pred = model(conti_input)
        
        loss = criterion(true_y, pred)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        total_loss.append(loss)
        
    return sum(total_loss)/len(total_loss)


adj = torch.tensor([[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], 
                    [0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                    [0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                    [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0],
                    [1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                    [1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                    [1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                    [1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                    [1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                    [1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0],
                    [1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0],
                    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0],
                    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0],
                    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0],
                    [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0],
                    [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]]
                ).float()

norm_adj = adj/adj.sum(dim=-1).unsqueeze(-1)
norm_adj = norm_adj.to(device)

deng = HSDSTM(
    adj=norm_adj,
    input_size=16,
    seq_len=48,
    num_channels=[16, 16],
    node_dim=1,
    dropout=0.1,
    num_levels=3,
    tau=12,
    num_quantiles=5
    )

In [11]:

deng.to(device)
optimizer = optim.Adam(deng.parameters(), lr=0.001)

criterion = QuantileRisk(12, [0.1, 0.3, 0.5, 0.7, 0.9], 1, device)
train_dataset = TensorDataset(torch.FloatTensor(train_input_deng), torch.FloatTensor(train_label))
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=512)

pbar = tqdm(range(100))

for epoch in pbar:        
    train_loss = train(deng, train_loader, criterion, optimizer, device)
    pbar.set_description("Train Loss: {:.4f}".format(train_loss))

torch.save(deng.state_dict(), '../assets/HSDSTM.pth')

  0%|          | 0/1 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
def generate_eval_ts(df, label_df, input_seq_len=48, tau=12):    
    col_labels =  ['wl_1018680'] # ['wl_1018662', 'wl_1018680', 'wl_1018683', 'wl_1019630']
    
    tmp_df = np.array(df.loc[df['year'] == 2021, :])
    tmp_label_df = np.array(label_df.loc[label_df['year'] == 2021, col_labels])
    
    n = tmp_df.shape[0] - input_seq_len - tau 
    
    tmp_conti_input = tmp_df[:, 4:] # (4416, 16)
    conti_input = np.zeros((n, input_seq_len, tmp_conti_input.shape[1]), dtype=np.float32)
    label = np.zeros((n, tau, len(col_labels)))

    past_input = np.zeros((n, input_seq_len, len(col_labels)), dtype=np.float32)
    label = np.zeros((n, tau, len(col_labels)))

    for j in range(n):
        past_input[j, :, :] = tmp_label_df[j:(j+input_seq_len), :]/1000
        conti_input[j, :, :] = tmp_conti_input[j:(j+input_seq_len), :]
        label[j, :, :] = tmp_label_df[(j+input_seq_len):(j+input_seq_len+tau), :]/1000

    return conti_input, label, past_input

In [ ]:
test_input, eval_label, past_input = generate_eval_ts_for_deng(df_test_total, df_merged)

deng = HSDSTM(
    adj=adj/adj.sum(dim=-1).unsqueeze(-1),
    input_size=16,
    seq_len=48,
    num_channels=[16, 16],
    node_dim=1,
    dropout=0.1,
    num_levels=3,
    tau=12,
    num_quantiles=5
    )

deng.load_state_dict(torch.load('../assets/HSDSTM.pth',  map_location="cpu"))
deng.eval()

eval_label = torch.tensor(eval_label)
deng_output = deng(torch.tensor(test_input))

torch.maximum(0.9 * (eval_label.squeeze() - deng_output[..., 4].squeeze()), (1-0.9)*(deng_output[..., 4].squeeze() -eval_label.squeeze() )).mean() # 0.0030
torch.maximum(0.7 * (eval_label.squeeze() - deng_output[..., 3].squeeze()), (1-0.7)*(deng_output[..., 3].squeeze() -eval_label.squeeze() )).mean() # 0.0051
torch.maximum(0.5 * (eval_label.squeeze() - deng_output[..., 2].squeeze()), (1-0.5)*(deng_output[..., 2].squeeze() -eval_label.squeeze() )).mean() # 0.0053

np.mean(eval_label.squeeze().cpu().numpy() < deng_output[..., 4].squeeze().detach().cpu().numpy())
0.9 - np.mean(eval_label.squeeze().cpu().numpy() < deng_output[..., 4].squeeze().detach().cpu().numpy())

np.mean(eval_label.squeeze().cpu().numpy() < deng_output[..., 3].squeeze().detach().cpu().numpy())
0.7 - np.mean(eval_label.squeeze().cpu().numpy() < deng_output[..., 3].squeeze().detach().cpu().numpy())

np.mean(eval_label.squeeze().cpu().numpy() < deng_output[..., 2].squeeze().detach().cpu().numpy())
0.5 - np.mean(eval_label.squeeze().cpu().numpy() < deng_output[..., 2].squeeze().detach().cpu().numpy())

In [ ]:
import seaborn as sns
from ing_theme_matplotlib import mpl_style # pip install ing_theme_matplotlib 
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams["figure.dpi"] = 300
mpl_style(dark=False)

gtcn_output = deng.levels[0][0](torch.tensor(test_input))

Wh = torch.matmul(gtcn_output, deng.levels[0][1].gat.W)

e = deng.levels[0][1].gat._prepare_attentional_mechanism_input(Wh)
zero_vec = -9e15*torch.ones_like(e)
attention = torch.where(deng.adj > 0, e, zero_vec)
alpha_ = F.softmax(attention, dim=-1)

### Rainy
batch_num = 1549
time_step = 1
plt.matshow(alpha_[batch_num, time_step, ...].detach().numpy(), cmap='Reds')
plt.colorbar()
plt.xlabel("Variable index")
plt.ylabel("Variable index")

### Dry
batch_num = 60
time_step = 0
plt.matshow(alpha_[batch_num, time_step, ...].detach().numpy(), cmap='Reds')
plt.colorbar()
plt.xlabel("Variable index")
plt.ylabel("Variable index")